# Importing HLA frequency files from NMDP.org to obtain most frequent alleles

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
HLAlist = [] # initiate
for x in 'ABC': # HLA frequency files from NMDP.org are organised per HLA allele; HLA-A, HLA-B, HLA-C
    df = pd.read_excel("/Users/pcevaal/Desktop/TheoreticalBiol/%s.xlsx" %x) # open corresponding NMDP.org file
    df_sort = df.sort_values(['CAU_rank'], ascending = True) # For general studies, select Caucasian population
    top20 = df_sort.head(20) # select top20 most frequent alleles in this population
    HLAs = top20[x] # generate list of alleles (are in column named A/B/C, depending on which file is opened)
    list = [HLA.encode() for HLA in HLAs] # transform from unicode
    HLAlist += list # merge most frequent HLA-A + HLA-B + HLA-C alleles into one list
sorted(HLAlist) # sort

Replace with below section to change human population to base most frequent alleles on + to change the top .. of alleles to be taken into further analysis. Currently set to Caucasian population and top20 alleles per HLA gene (ABC). 

In [ ]:
df.sort_values(['%s_rank' %raw_input("population:")], ascending = False) # if we would want to base our analysis on the most frequent alleles in another population. Not used in this project
top20 = df.head(int(raw_input("no of rows:"))) # if we were to use a different number of most frequent alleles. Not used in this project

Rewrite HLAlist into format that can be used as netMHCpan input file ('HLAinput.txt')

In [2]:
MHC = '' # initiate
for x in HLAlist: # for each allele, currently in format eg A*24:03g
    MHC += 'HLA-' + x[0] + x[2:7] + ','
    # writes in correct format, removes 'g' annotation at the end
with open('HLAinput.txt', 'w') as x:
            x.write(MHC[:-1]) # delete final ','

# Collect additional informaton on HLA alleles for further analysis

As for now chosen to use top20 most frequent alleles per type (A/B/C) for input in netMHCpan. See above on how to change.  
For graphical analysis however easier to quickly leave out some alleles, therefore store frequency rank within final_output library. 

In [70]:
HLAlist = [] # initiate
HLAranks = {}
iter = 0
for x in 'ABC': 
    df = pd.read_excel("/Users/pcevaal/Desktop/TheoreticalBiol/%s.xlsx" %x) # open corresponding NMDP.org file
    df_sort = df.sort_values(['CAU_rank'], ascending = True) # For general studies, select Caucasian population
    top20 = df_sort.head(20) # select top20 most frequent alleles in this population
    HLAs = top20[x] # generate list of alleles (are in column named A/B/C, depending on which file is opened)
    iter = 1 # initiate counter that will indicate position of allele in top20
    for HLA in HLAs:
        MHC = 'HLA-' + HLA[0] + HLA[2:7] # write in correct format
        HLAranks[MHC.encode()] = iter # generate dict item with allele name and corresponding frequency position
        #print iter, HLA
        iter += 1
%store HLAranks

Stored 'HLAranks' (dict)


Secondly, we wish to analyse results in correlation to known protective or destructive characteristics of HLA alleles. Therefore, quantify / assign protection score based on most recent literature paper on this topic (see below)

source info on HLA supertypes: http://www.ufrgs.br/imunovet/molecular_immunology/hla.html  
source literature: Sundaramurthi 2017

In [ ]:
# input information from literature on protective HLA serotypes and/or genotypes
literature_protective = ['HLA-A01', 'HLA-A02:01', 'HLA-A68:02', 'HLA-A02:02', 'HLA-A02:05', 'HLA-A02:14', 'HLA-A11', 'HLA-A11:01', 'HLA-A25', 'HLA-A32', 'HLA-A74:01', 'HLA-B13:02', 'HLA-C08:02', 'HLA-B18', 'HLA-B27', 'HLA-B27:05', 'HLA-B51', 'HLA-B52', 'HLA-B57', 'HLA-B57:01', 'HLA-B57:03','HLA-B15', 'HLA-C06:02', 'HLA-C07:01']
print literature_protective

In [ ]:
# input information from literature on detrimental HLA serotypes and/or genotypes
literature_detrimental = ['HLA-A03', 'HLA-A19', 'HLA-A23', 'HLA-23:01', 'HLA-A24', 'HLA-B07', 'HLA-B07:02', 'HLA-B18', 'HLA-B35', 'HLA-B35:03', 'HLA-B35:02', 'HLA-B35:03', 'HLA-B35:04', 'HLA-B53:01', 'HLA-B37', 'HLA-B40', 'HLA-B40:06', 'HLA-B42:01', 'HLA-B44', 'HLA-B49', 'HLA-B53', 'HLA-B53:01', 'HLA-C04', 'HLA-C07']
print literature_detrimental

Strategy:  
If allele serotype is not found in either literature_protective or literature_destructive, assign 'unknown'  
If allele serotype is found in both literature_protective and literature_destructive, assign 0  
If allele genotype if found in literature_protective, assign +10  
If allele genotype if found in literature_destructive, assign -10  
If allele genotype is not found in literature_protective, but allele serotype is, assign +5  
If allele genotype is not found in literature_destructive, but allele serotype is, assign -5

In [113]:
col1_HLAs = sorted(HLAranks.keys()) # generate list of most frequent alleles, sorted on alphabetical order 

protection_score = {} # initiate
for HLA in col1_HLAs: 
    protection_score[HLA] = 0 # initiate item in dictionary for all alleles from top20

for HLA in protection_score: # loop through newly generated dict
    if HLA in literature_protective: # if entire HLA name is known in literature, i.e. genotype is known
        protection_score[HLA] += 10
    elif HLA[0:7] in literature_protective: # if only HLA serotype is known
        protection_score[HLA] += 5
    if HLA in literature_detrimental: # idem for detrimental
        protection_score[HLA] -= 10
    elif HLA[0:7] in literature_detrimental:
        protection_score[HLA] -= 5
    if HLA[0:7] in literature_detrimental and HLA[0:7] in literature_protective: # one allele has ambigious information in literature, is both protective and detrimental? 
        protection_score[HLA] = 0
    if not HLA[0:7] in literature_detrimental and not HLA[0:7] in literature_protective: # if no info at all is known
        protection_score[HLA] = 'unknown'
print pd.DataFrame(protection_score.items()).sort_values(0)

%store protection_score


             0        1
28  HLA-A01:01        5
38  HLA-A02:01  unknown
36  HLA-A02:05  unknown
49  HLA-A03:01       -5
9   HLA-A11:01       10
11  HLA-A23:01       -5
43  HLA-A24:02       -5
10  HLA-A25:01        5
23  HLA-A26:01  unknown
6   HLA-A29:01  unknown
50  HLA-A29:02  unknown
33  HLA-A30:01  unknown
31  HLA-A30:02  unknown
56  HLA-A31:01  unknown
40  HLA-A32:01        5
14  HLA-A33:01  unknown
13  HLA-A33:03  unknown
18  HLA-A66:01  unknown
44  HLA-A68:01  unknown
30  HLA-A68:02  unknown
42  HLA-B07:02      -10
4   HLA-B08:01  unknown
29  HLA-B13:02  unknown
15  HLA-B14:02  unknown
51  HLA-B15:01        5
3   HLA-B18:01        0
32  HLA-B27:05       10
35  HLA-B35:01       -5
39  HLA-B35:02      -10
37  HLA-B35:03      -10
17  HLA-B37:01       -5
26  HLA-B38:01  unknown
12  HLA-B40:01       -5
25  HLA-B40:02       -5
5   HLA-B44:02       -5
24  HLA-B44:03       -5
0   HLA-B49:01       -5
55  HLA-B51:01        5
19  HLA-B55:01  unknown
22  HLA-B57:01       10
53  HLA-C01:02  

In [6]:
HLAknown = list(protection_df.loc[protection_df[1]!= 'unknown'][0]) # exclude HLA alleles for which no information is available on protection
%store HLAknown
print ",".join(HLAknown) # only use these alleles for netMHCpan, this prints these alleles in right format

HLA-B35:03,HLA-B35:02,HLA-B07:02,HLA-B35:01,HLA-A03:01,HLA-C04:01,HLA-B40:01,HLA-C07:04,HLA-B40:02,HLA-C07:02,HLA-B37:01,HLA-B49:01,HLA-A23:01,HLA-B44:02,HLA-B44:03,HLA-A24:02,HLA-B18:01,HLA-A25:01,HLA-A32:01,HLA-B51:01,HLA-A01:01,HLA-C07:01,HLA-B15:01,HLA-A11:01,HLA-B57:01,HLA-B27:05


# Coevolution

In [2]:
# For coevolution studies we will compare multiple populations, 
# and thus will need to generate lists of most frequent HLA alleles in those populations
# Initially we will only focus on HLA-A and HLA-B 
def HLA_most_freq(population, topx): 
    HLAlist = []
    for x in 'AB': # HLA frequency files from NMDP.org are organised per HLA allele; HLA-A, HLA-B
        df = pd.read_excel("/Users/pcevaal/Desktop/TheoreticalBiol/%s.xlsx" %x) # open corresponding NMDP.org file
        df_sort = df.sort_values(['%s_rank' %population], ascending = True) # sort dataframe on frequencies of desired population
        top = df_sort.head(topx) # select top20 most frequent alleles in this population
        HLAs = top[x] # generate list of alleles (are in column named A/B/C, depending on which file is opened)
        list = [HLA.encode() for HLA in HLAs] # transform from unicode
        HLAlist += list # merge most frequent HLA-A + HLA-B alleles into one list
    HLAlist.sort()
    MHC = ''
    for x in HLAlist:
        MHC += 'HLA-' + x[0] + x[2:7] + ','
        # writes in correct format, removes 'g' annotation at the end
    MHClist = MHC.split(',')
    del MHClist[-1] # remove final item in list, which is empty (after last ',')
    return MHClist

Globally, HIV1-B and HIV1-C are widely distributed but overrepresented in distinguished parts of the world. One of the hypotheses is that over time, HIV clades and human HLA alleles have co-evolved (a form of host-pathogen interaction / coevolution). We therefore analyse the two main HIV1 strains and their corresponding dominant populations:   
HIV1-B is the dominant strain in Europe    
HIV1-C is the dominant strain in Central/Southern Africa, and India (among others) (AFB and AINDI populations)   

The NMDP.org database is based solely on the population of the USA, however does provide information on the ethnical background of the blood donors. Assuming the HLA allele frequencies will not have evolved so quickly after migration to the USA, we will use the most frequent HLA alleles in the American subpopulations as a representative of the global populations:   
Europe: EURCAU population   
India: AINDI population  
Africa: AFB population

In [3]:
# Generate lists of top4 most frequent HLA-A and HLA-B alleles for three study populations
AFB_4 = HLA_most_freq('AFB', 4)
EURCAU_4 = HLA_most_freq('EURCAU', 4)
AINDI_4 = HLA_most_freq('AINDI', 4)

In [3]:
# Organise top alleles into dictionary
HLAfreqs_for_coev = {}
HLAfreqs_for_coev['AFB'] = AFB_4
HLAfreqs_for_coev['EURCAU'] = EURCAU_4
HLAfreqs_for_coev['AINDI'] = AINDI_4
%store HLAfreqs_for_coev

Stored 'HLAfreqs_for_coev' (dict)


In [14]:
newMHCpan = AFB_4 + EURCAU_4 + AINDI_4
print ",".join(list(set(newMHCpan))) # Write list in proper format for input in netMHCpan

HLA-A33:03,HLA-B53:01,HLA-A23:01,HLA-B15:01,HLA-B40:06,HLA-A24:02,HLA-B52:01,HLA-B15:03,HLA-B44:02,HLA-A03:01,HLA-B35:01,HLA-A01:01,HLA-B08:01,HLA-A02:01,HLA-B51:01,HLA-B44:03,HLA-A11:01,HLA-B07:02,HLA-A30:01


In [26]:
%store -r HIVproteome
# select only HIV1 reference strains for coevolution studies, 
# as HIV1 and HIV2 differ too much to sensibly study coevolution across all clades
for protein in HIVproteome:
    ref = ("Ref.A1", "Ref.B", "Ref.C")
    with open("/Users/pcevaal/Desktop/TheoreticalBiol/HIV12_%s.fasta" %protein, 'r') as f:
        HIV12 = f.read().split('>') # use HIV12 merged fasta files for each protein in HIVproteome
    HIV1ABClist = [">" + strain for strain in HIV12 if strain.startswith(ref)] # select those fasta inputs from HIV1
    HIV1ABC = "".join(HIV1ABClist) # join back into one list
    with open("/Users/pcevaal/Desktop/TheoreticalBiol/HIV1ABC_%s.fasta" %protein, 'w') as g:
        g.write(HIV1ABC) # write new fasta file, to be used in netMHCpan 

Run netMHCpan again with HLAs from newMHCpan set, and only strain sequences from HIV1A1, HIV1B, HIV1C clades

### Enriched alleles per population instead of purely most frequent

Above study is based on most frequent alleles in each of the three study populations. However, these alleles are in fact quite overlapping, which could explain the lack of differences observed in the presentation of peptides from the two HIV1 clades.   
Therefore, we will now assess presentation for those HLA alleles that are enriched in the study populations rather than merely frequently present. These enriched alleles would better represent alleles that might have co-evolved than alleles that are perhaps often present in all human being across the entire glove. 

In [121]:
# Make formula for each population to calculate the difference in allele frequency:
# for each allele, the rank for one population is compared to that for the other two populations, 
# and those alleles with a great difference in rank (meaning a strong enrichment) are then selected
def AFB_dif(y):
    return (newdf.loc[y,'AINDI_rank']-newdf.loc[y,'AFB_rank'])+(newdf.loc[y,'EURCAU_rank']-newdf.loc[y,'AFB_rank'])
def AINDI_dif(y):
    return (newdf.loc[y,'AFB_rank']-newdf.loc[y,'AINDI_rank'])+(newdf.loc[y,'EURCAU_rank']-newdf.loc[y,'AINDI_rank'])
def EURCAU_dif(y):
    return (newdf.loc[y,'AINDI_rank']-newdf.loc[y,'EURCAU_rank'])+(newdf.loc[y,'AFB_rank']-newdf.loc[y,'EURCAU_rank'])
def to_list(inputseries):
    MHC = []
    for x in inputseries:
        MHC.append('HLA-' + x.encode()[0] + x.encode()[2:7])
    return MHC   
    
HLAlist = []
for x in 'AB': # HLA frequency files from NMDP.org are organised per HLA allele; HLA-A, HLA-B
    df = pd.read_excel("/Users/pcevaal/Desktop/TheoreticalBiol/%s.xlsx" %x) # open corresponding NMDP.org file
    newdf = df.loc[:,[x,'AFB_rank','AINDI_rank','EURCAU_rank']] # select sensible columns from NMDP.org file
    # some alleles are not at all present in a certain population, which yields a 'NaN' that would give an error in our code
    # therefore, replace these with the highest rank possible for that population
    row = newdf.loc[0] # select an example row in which the 'NaN' is present
    nan = row[1] # this entry in the dataframe contains the 'NaN', manual entry does not work
    if x == 'A':
        newdf.loc[:,'AFB_rank'].replace(nan, 151, inplace=True) # replace all 'NaN's for HLA-A in AFB pop with rank 151 (max is 150)
        newdf.loc[:,'AINDI_rank'].replace(nan, 168, inplace=True) # replace all 'NaN's for HLA-A in AINDI pop with rank 168 (max is 167)
        newdf.loc[:,'EURCAU_rank'].replace(nan, 318, inplace=True) # replace all 'NaN's for HLA-A in EURCAU pop with rank 318 (max is 317)
        for y in range(len(newdf)): # for each allele, via its y coordinate,   
            if newdf.loc[y,'AFB_rank'] != 151: # if in that population the allele was present (!= max rank)
                newdf.loc[y,'AFB_dif'] = AFB_dif(y) # calculate difference score
            if newdf.loc[y,'AINDI_rank'] != 168:
                newdf.loc[y,'AINDI_dif'] = AINDI_dif(y)
            if newdf.loc[y,'EURCAU_rank'] != 318:
                newdf.loc[y,'EURCAU_dif'] = EURCAU_dif(y)
        # generate top10 of enriched alleles in each population by sorting (descending) on difference score
        A_EURCAU = to_list(newdf.sort_values('EURCAU_dif',ascending=False).head(10).loc[:,'A'])
        A_AFB = to_list(newdf.sort_values('AFB_dif',ascending=False).head(10).loc[:,'A'])
        A_AINDI = to_list(newdf.sort_values('AINDI_dif',ascending=False).head(10).loc[:,'A'])
    if x == 'B':
        newdf.loc[:,'AFB_rank'].replace(nan, 198, inplace=True) # replace all 'NaN's for HLA-B in AFB pop with rank 198 (max is 197)
        newdf.loc[:,'AINDI_rank'].replace(nan, 243, inplace=True) # replace all 'NaN's for HLA-B in AINDI pop with rank 243 (max is 242)
        newdf.loc[:,'EURCAU_rank'].replace(nan, 531, inplace=True) # replace all 'NaN's for HLA-B in EURCAU pop with rank 531 (max is 530)
        for y in range(len(newdf)): # see above for x == 'A'
            if newdf.loc[y,'AFB_rank'] != 198:
                newdf.loc[y,'AFB_dif'] = AFB_dif(y)
            if newdf.loc[y,'AINDI_rank'] != 243:
                newdf.loc[y,'AINDI_dif'] = AINDI_dif(y)
            if newdf.loc[y,'EURCAU_rank'] != 531:
                newdf.loc[y,'EURCAU_dif'] = EURCAU_dif(y)
        B_EURCAU = to_list(newdf.sort_values('EURCAU_dif',ascending=False).head(10).loc[:,'B'])
        B_AFB = to_list(newdf.sort_values('AFB_dif',ascending=False).head(10).loc[:,'B'])
        B_AINDI = to_list(newdf.sort_values('AINDI_dif',ascending=False).head(10).loc[:,'B'])


In [127]:
# join lists of enriched HLA-A and -B alleles for each population
AFB = A_AFB+B_AFB
AINDI = A_AINDI+ B_AINDI
EURCAU = A_EURCAU+ B_EURCAU

enriched_alleles_perpop = {} # merge these lists in a dictionary
enriched_alleles_perpop['AFB'] = AFB
enriched_alleles_perpop['AINDI'] = AINDI
enriched_alleles_perpop['EURCAU'] = EURCAU
%store enriched_alleles_perpop

newMHCpan_enriched = list(set(AFB+AINDI+EURCAU)) # generate list of all unique HLA alleles that need to be run in netMHCpan
enriched_set =",".join(list(set(newMHCpan_enriched))) # write in appropriate format
print enriched_set

Stored 'enriched_alleles_perpop' (dict)
HLA-A01:12,HLA-A02:38,HLA-B44:04,HLA-A23:06,HLA-B44:07,HLA-A24:24,HLA-A02:10,HLA-B07:18,HLA-A02:13,HLA-A11:03,HLA-A02:16,HLA-B35:10,HLA-A02:19,HLA-A25:02,HLA-B14:05,HLA-A02:30,HLA-A11:44,HLA-B15:24,HLA-A33:05,HLA-A24:32,HLA-A32:04,HLA-B40:50,HLA-A68:10,HLA-B15:75,HLA-A24:13,HLA-A74:11,HLA-B15:52,HLA-B15:54,HLA-A31:12,HLA-B48:04,HLA-B56:22,HLA-B82:02,HLA-A11:19,HLA-B08:09,HLA-A30:25,HLA-B44:15,HLA-B13:03,HLA-A01:09,HLA-B44:09,HLA-B07:04,HLA-B35:27,HLA-A02:27,HLA-A23:15,HLA-A74:09,HLA-A32:07,HLA-B27:10,HLA-B27:14,HLA-B35:41,HLA-A24:04,HLA-B40:26,HLA-A31:16,HLA-B15:05,HLA-B40:23,HLA-A68:27,HLA-B56:04,HLA-B15:67,HLA-B15:47,HLA-B15:45,HLA-A26:30,HLA-B53:09


In [107]:
%store -r HIVproteome
for protein in HIVproteome: # in this substudy, only HIV1-B and 1-C are compared, thus exclude other refstrains from .fasta files
    ref = ("Ref.B", "Ref.C")
    with open("/Users/pcevaal/Desktop/TheoreticalBiol/HIV12_%s.fasta" %protein, 'r') as f:
        HIV12 = f.read().split('>')
    HIV1BClist = [">" + strain for strain in HIV12 if strain.startswith(ref)] # delete strain if not clade B or C
    HIV1BC = "".join(HIV1BClist)
    with open("/Users/pcevaal/Desktop/TheoreticalBiol/HIV1BC_%s.fasta" %protein, 'w') as g:
        g.write(HIV1BC) # write to new file, to be used as netMHCpan input